In [1]:
%%capture
!pip install pandas
!pip install plotly-express
!pip install statsmodels
!pip install tqdm


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels as sm 
from sklearn.metrics import r2_score
from tqdm import tqdm
import plotly.express as px
from sklearn.preprocessing import StandardScaler
# model imports
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm1
from sklearn.metrics import mean_squared_error, r2_score
import itertools
import math


# Creating Sarimax Model With Excog Variables At A Country Level

We will be creating a Sarimax model at country level because each time forecasting relies on the trends in data, therefore you cannot have multiple instances for a single time point. Moreover, as a simple model, it only requires not many datapoints to make predictions. Below, we detail the process that we have for our model.

1. Data Preperation
2. Hypertuning Model Parameters With Training and Validation Set
3. Testing model on holdout set


## Preparing Data for Arima
We have multple task to prepare data for ARIMA
### Data Wrangling
1. We have to cast our Year into a datetime object
2. We then have to set it as an index of the array
3. Finally, we have to set the period of the data to Yearly frequency

### Spliting Data into Training, Validation, and Holdout
We selection the following time periods.
```
Training (1750 - 1950)
Validation (1950 - 1980)
Testing (1980 - 2013)
```

### Scaling Data By Standard Scalar
The numerical method used for ARIMA involves BFGS, which is not
scale invariant, therfore we scale the x-features by that of
the training data

In [3]:
def get_country_df(country):
    df = pd.read_csv('../Data/global_average_yearly_temp_with_features_clean.csv')
    df = df[df["iso_code"] == country]
    df["Year_idx"] = pd.to_datetime(df.year, format="%Y")
    df = df.set_index("Year_idx")
    df.index = df.index.to_period("Y")
    return df

def get_test_train_valid(country, train_split = '1950', validation_split = '1980'):
    country_df = get_country_df(country)
    
    train = country_df.loc[:train_split]
    valid = country_df.loc[train_split: validation_split]
    test  = country_df.loc[validation_split:]
    
    return train, test, valid

def scale(train_x, valid_x, test_x):
    scaler = StandardScaler()
    model_x = pd.concat([train_x, valid_x])

    scaler.fit(model_x)
    model_x = scaler.transform(model_x)
    train_x = scaler.transform(train_x)
    valid_x = scaler.transform(valid_x)
    test_x  = scaler.transform(test_x)
    return train_x, valid_x, test_x, model_x

In [4]:
country = "FRA"
train, test, valid = get_test_train_valid(country)

train_x, train_y = train[train.columns[2:-2]], train["AvgYearlyTemp"]
test_x, test_y = test[test.columns[2:-2]], test["AvgYearlyTemp"]
valid_x, valid_y = valid[valid.columns[2:-2]], valid["AvgYearlyTemp"]

train_x, valid_x, test_x, model_x = scale(train_x, valid_x, test_x)
model_y = pd.concat([train_y, valid_y])

## Hypertuning Model Parameters With Training and Validation Set
We opt for an exhaustive grid search to find the parameters that give the best result. Here, we only use the following assumptions for the sarimax model. The external regressors are time varying, and have measurement error. 


In [5]:
def eval_arima_excog_walk_forward_validation_model(train_x, train_y, test_x, test_y, arima_order):
    # DO NOT USE THIS 

    history = [x for x in train_y]
    predictions = []
    
    for i in range(len(test_x)):
        model = ARIMA(history, exog = train_x, order = arima_order)
        model_fit = model.fit()
        
        y_hat = model_fit.forecast(exog = test_x.iloc[[i]])
        predictions.append(y_hat[0])
        
        train_x = pd.concat([train_x, test_x.iloc[[i]]], axis = 0)
        history.append(test_y[i])

    rmse = math.sqrt(mean_squared_error(test_y, predictions))
    return model_fit, rmse, predictions

def eval_arima_excog(train_x, train_y, test_x, test_y, arima_order, trend = 'c'):
    # DO NOT USE THIS 
    model = ARIMA(train_y, exog = train_x, order = arima_order, trend = trend)
    model_fit = model.fit()
    predictions = model_fit.forecast(len(test_x), exog = test_x)
    rmse = math.sqrt(mean_squared_error(test_y, predictions))
    return model_fit, rmse, predictions

def eval_sarimax_excog(train_x, train_y, test_x, test_y, arima_order):
    # USE THIS
    model = sm1.tsa.statespace.SARIMAX(train_y, exog = train_x, order = arima_order, \
                                       time_varying_regression = True, mle_regression = False, measurement_error = True)
    model_fit = model.fit()
    predictions = model_fit.forecast(len(test_x), exog = test_x)
    rmse = math.sqrt(mean_squared_error(test_y, predictions))
    predictions.index = test_y.index
    return model_fit, rmse, predictions

def eval_excog_models(train_x, train_y, test_x, test_y, p_values, d_values, q_values, f):
    arima_orders = itertools.product(*[p_values, d_values, q_values])
    arima_orders = list(arima_orders)
    best_order, best_score = None, float("inf") 
    
    count = 0
    for arima_order in tqdm(arima_orders):
        try:
            _, rmse, _ = f(train_x, train_y, test_x, test_y, arima_order)
            
            if rmse < best_score:
                best_score, best_order = rmse, arima_order
                print(f"ARIMA RMSE = {best_score}")
        except Exception as e:
            print(e)
            pass
        count += 1
    print('DONE')
    return best_order
    

In [6]:
p_values = [0, 1, 2, 4]
d_values = range(0, 3)
q_values = range(0, 3)
best = eval_excog_models(train_x, train_y, valid_x, valid_y, p_values, d_values, q_values, eval_sarimax_excog)
print(best)

  0%|                                                                                            | 0/36 [00:00<?, ?it/s] This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           14     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.19161D+00    |proj g|=  2.00451D-01

At iterate    5    f=  1.10007D+00    |proj g|=  7.37511D-01

At iterate   10    f=  1.02092D+00    |proj g|=  1.08452D-01

At iterate   15    f=  1.00944D+00    |proj g|=  4.43219D-02

At iterate   20    f=  1.00373D+00    |proj g|=  4.39403D-02

At iterate   25    f=  9.97262D-01    |proj g|=  6.95343D-02

At iterate   30    f=  9.80300D-01    |proj g|=  9.99451D-02
  ys=-3.159E-06  -gs= 1.949E-06 BFGS update SKIPPED



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate   35    f=  9.79888D-01    |proj g|=  1.65039D-01
  ys=-6.700E-10  -gs= 2.584E-09 BFGS update SKIPPED

At iterate   40    f=  9.79886D-01    |proj g|=  1.34142D-01



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.


...


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   20     39    195      3     2     0   1.478D-01   9.955D-01
  F =  0.99554848432130683     

ABNORMAL_TERMINATION_IN_LNSRCH                              
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           21     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.26222D+00    |proj g|=  1.83801D-01


 This problem is unconstrained.



At iterate    5    f=  1.15112D+00    |proj g|=  7.39937D-01

At iterate   10    f=  1.06634D+00    |proj g|=  1.33959D-01

At iterate   15    f=  1.03445D+00    |proj g|=  9.98667D-02

At iterate   20    f=  9.97530D-01    |proj g|=  1.61477D-01

At iterate   25    f=  9.95731D-01    |proj g|=  8.83632D-02

At iterate   30    f=  9.95519D-01    |proj g|=  1.09238D-01

At iterate   35    f=  9.95503D-01    |proj g|=  5.52107D-02

At iterate   40    f=  9.95485D-01    |proj g|=  1.31976D-01

At iterate   45    f=  9.95474D-01    |proj g|=  6.36860D-02
  ys=-5.675E-05  -gs= 6.280E-06 BFGS update SKIPPED


/home/sumants/miniconda3/envs/cis545/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
100%|███████████████████████████████████████████████████████████████████████████████████| 36/36 [01:27<00:00,  2.43s/it]


At iterate   50    f=  9.95471D-01    |proj g|=  9.10169D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   21     50    148      1     1     0   9.102D-02   9.955D-01
  F =  0.99547105578843109     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
DONE
(1, 2, 1)


## Testing on the holdout
We now use the best parameters from an exhaustive grid search to predict on our holdout set

In [16]:
def plot(df, country, layout, line1 = 'AvgYearlyTemp', line2 = 'Predictions', x = 'year', error = 'AvgTempUncertainty'):
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go


    fig = go.Figure(layout = layout)
    trace_1 = go.Line(name = f"Actual {country} Avg Yearly Temp", 
                     x = test[x],
                     y = test[line1],
                     error_y = dict(type='data', array = test[error]))


    trace_2 = go.Line(name = f"Predictions of {country} Avg Yearly Temp", 
                     x = test[x],
                     y = test[line2])

    fig.add_trace(trace_1)
    fig.add_trace(trace_2)
    
    return fig

In [7]:
model_fit, rmse, predictions = eval_sarimax_excog(model_x, model_y, test_x, test_y, best)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           17     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.39543D+00    |proj g|=  1.63774D-01

At iterate    5    f=  1.05812D+00    |proj g|=  1.84565D-01

At iterate   10    f=  1.01208D+00    |proj g|=  1.86223D-01

At iterate   15    f=  9.99406D-01    |proj g|=  7.92741D-02

At iterate   20    f=  9.77967D-01    |proj g|=  7.75493D-02

At iterate   25    f=  9.76222D-01    |proj g|=  9.19232D-02

At iterate   30    f=  9.75887D-01    |proj g|=  3.63840D-02

At iterate   35    f=  9.75817D-01    |proj g|=  8.45809D-02

At iterate   40    f=  9.75753D-01    |proj g|=  2.81821D-02

At iterate   45    f=  9.75655D-01    |proj g|=  4.00342D-02

At iterate   50    f=  9.75533D-01    |proj g|=  5.62488D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/home/sumants/miniconda3/envs/cis545/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [13]:
%%capture
test["Predictions"] = predictions


In [17]:
layout = dict(xaxis=dict(title='Year'),
              yaxis=dict(title='Temperature'))
fig = plot(test, "France", layout)
fig.show()